In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms


In [9]:
class AlexNetModel(nn.Module):
    def __init__(self, num_classes=38):
        super(AlexNetModel, self).__init__()
        
        # Define data augmentation and normalization layers
        self.transform_train = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(227),
            transforms.ToTensor(),  # Convert PIL image to tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.transform_test = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(227),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        # Define convolutional layers
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2, groups=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1, groups=2)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1, groups=2)
        
        # Define normalization and pooling layers
        self.norm1 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75)
        self.norm2 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.pool5 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        # Define fully connected layers
        self.fc6 = nn.Linear(256 * 6 * 6, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, num_classes)
        
        # Define dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        # Data augmentation and normalization
        if self.training:
            x = self.transform_train(x)
        else:
            x = self.transform_test(x)
        
        # Convolutional layers with ReLU and normalization
        x = F.relu(self.conv1(x))
        x = self.norm1(x)
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.norm2(x)
        x = self.pool2(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.pool5(x)
        
        # Flatten and pass through fully connected layers with dropout
        x = x.view(-1, 256 * 6 * 6)
        x = F.relu(self.fc6(x))
        x = self.dropout(x)
        x = F.relu(self.fc7(x))
        x = self.dropout(x)
        x = self.fc8(x)
        
        return x

In [10]:
model = AlexNetModel()

AttributeError: module 'torchvision.transforms' has no attribute 'Resize'